# Open-circuit track down in a coaxial system

## Introduction

In this demo we want to spot an open-circuit in a coaxial system as the one in the figure:

![Coaxial demo scheme](./coaxial-scheme.png)

The four oscilloscopes we aim to control are the Agilent DSOX1102A and the arbitrary function generator is the Tektronix AFG3252.

As you can see in the oscilloscopes screen, a backwards signal is expected to appear due to the open circuit, as this means a reflection point with the following reflection coefficent:

$\Gamma = \frac{Z_L - Z_0}{Z_L + Z_0}$

Where $Z_0$ is the characteristic impedance of the wire and $Z_L$ is the load impedante at this end point, in this case $\infty$. Thus, $\Gamma = 1$. Additionaly, since $Z_0 = 50\>Ω$, the resistors placed at the end of each branch avoid the emergence of other backwards signals.

So, we are measuring the time delay between the forwards and the backwards pulse in each oscilloscope and then we will find the location of the short circuit.

> If you are using this code in your own system, you will have to change the VISA addresses and maybe the IPs. Also if the instrumentation is not the same the SCPI commands used here propably won`t work with yours. Try to find them in the instrument documentation.

## First steps

Firstly, we need the gateways' IP and communication port and the instruments VISA addresses

In [7]:
IP1 = '192.168.1.11'; port1 = 50001
IP2 = '192.168.1.12'; port2 = 50002
IP3 = '192.168.1.13'; port3 = 50003
IP4 = '192.168.1.14'; port4 = 50004

AFG = 'USB0::0x0699::0x0345::C022028::INSTR' #This is attached to the first gateway
#Each osc is connected to the gateway of its number
osc1 = 'USB0::0x2A8D::0x1797::CN58446436::INSTR'
osc2 = 'USB0::0x2A8D::0x1797::CN58446449::INSTR'
osc3 = 'USB0::0x2A8D::0x1797::CN58446453::INSTR'
osc4 = 'USB0::0x2A8D::0x1797::CN59176280::INSTR'

IPs = [IP1, IP2, IP3, IP4]
ports = [port1, port2, port3, port4]
oscs = [osc1, osc2, osc3, osc4]

Also, the IPs, the ports and the oscilloscopes are listed, to simplify the code.

To control the instrumentation we are using `instin`, our wireless instrumentation control Python package. Also the `time` is needed to synchornize with the gateways.

In [2]:
#import instin
import time
import numpy as np

The first thing to do with the gateways is to synchronize them. This is easy using the `sicro` function of `instin`

In [3]:
print('PC time: ' + time.ctime(time.time()))

PC time: Wed Jan 20 10:45:09 2021


In [4]:
instin.sincro(IP1, port1)

In [5]:
instin.sincro(IP2, port2)

In [6]:
instin.sincro(IP3, port3)

In [7]:
instin.sincro(IP4, port4)

Now instruments can be opened

In [8]:
AFG = int(instin.open_inst(IP1, port1, AFG))

for i in range(len(IPs)):
    oscs[i] = int(instin.open_inst(IPs[i], ports[i], oscs[i]))

Instrument USB0::0x0699::0x0345::C022028::INSTR opened on 192.168.1.11
Instrument USB0::0x2A8D::0x1797::CN58446436::INSTR opened on 192.168.1.11
Instrument USB0::0x2A8D::0x1797::CN58446449::INSTR opened on 192.168.1.12
Instrument USB0::0x2A8D::0x1797::CN58446453::INSTR opened on 192.168.1.13
Instrument USB0::0x2A8D::0x1797::CN59176280::INSTR opened on 192.168.1.14


## Measurement process

Starting with the measurement process, the first thing to do is to set the AFG. To clearly see the forwards and backwars signal the best thing to do is to send a ultra sort pulse with a relative low frequency, avoiding the overlap between the backward pulse and the next forward pulse coming from the AFG.

In [9]:
freq_ret = '1e6'
duty = '10ns'
vpp = '1'
offset = '500mV'

Which are easy established with the `write` operation

In [10]:
instin.write(IP1, port1, AFG, 'outp 0')
instin.write(IP1, port1, AFG, 'func puls') #Setting the AFG to send pulses
instin.write(IP1, port1, AFG, 'freq '+ freq_ret) #With the correct freq
instin.write(IP1, port1, AFG, 'volt:ampl '+ vpp + ' vpp') #Setting the signal to be between 0 and 1
instin.write(IP1, port1, AFG, 'volt:offs '+ offset)
instin.write(IP1, port1, AFG, 'puls:widt '+ duty) #Using the minimum duty-cycle
instin.write(IP1, port1, AFG, 'outp 1') #Turning on the output

The signal in the coaxial cable is ready to be measured, let's set the osciloscopes. First we are using the autoset feature to aproximate to the visualization we want, then the time scale is set to be the same as the pulse and finaly we desplace the time position to de left in other to see both the forward and backward signals

This can be done one osciloscope after each other:

In [11]:
for i in range(len(oscs)):
    instin.write(IPs[i], ports[i], oscs[i], 'aut')
    instin.write(IPs[i], ports[i], oscs[i], 'acq:type aver')
    instin.write(IPs[i], ports[i], oscs[i], 'acq:count 128')
    instin.write(IPs[i], ports[i], oscs[i], 'tim:scal 20 ns')
    instin.write(IPs[i], ports[i], oscs[i], 'tim:pos 60 ns')

This can cause the oscilloscopes to freeze, due to the time that takes the autoset functionality. 

>This also happens when the oscilloscope is USB directly controlled, by the way

If this occurs, one solution is to send one order after each other, taking advantage of the time delay in the wireless connections.

In [12]:
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'aut')
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'acq:type aver')
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'acq:count 128')
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'tim:scal 20 ns')
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'tim:pos 60 ns')

Once the AFG and the oscilloscopes are set, the time delay in each oscilloscope can be measured. Since the measurements aviable in this oscilloscope are limited, we need to measure the delay with the `xmax` measure. To measure the backward pulse the time offset is used to hide the first pulse.

In [13]:
t = [0]*len(oscs)

for i in range(len(oscs)): 
    t[i] = float(instin.query(IPs[i], ports[i], oscs[i], 'meas:xmax? chan1'))
for i in range(len(oscs)):
    for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'tim:pos 140 ns')
for i in range(len(oscs)):
    t[i] = float(instin.query(IPs[i], ports[i], oscs[i], 'meas:xmax? chan1')) - t[i]

for i in range(len(oscs)):
    print('Osc{} delay (s) = {}'.format(i+1, t[i]))

Osc1 delay (s) = 1.1690000000000001e-07
Osc2 delay (s) = 1.075e-07
Osc3 delay (s) = 9.910000000000001e-08
Osc4 delay (s) = 9.87e-08


## Open circuit position estimation

With these delays, the short circuit position can be estimated. The time delay appearing in the oscilloscopes measuring the branches after the branch with the open circuit is always the same, $\Delta t = 2\frac{x_{op}}{v}$, where $x_{op}$ is the lenght from the branch union with the main line (A) and the short circuit. In the other hand, the time delay appearing in the oscilloscopes of the branches before the open-circuit is $\Delta t = 2\frac{x_{op} + x_{off}}{v}$, where $x_{off}$ is the distance between the connection point of the branch and A.

To estimate the short circuit position, we use the coaxial cable propagtion velocity (previously chracterized), $v = 0.7c$, (where $c$ is the speed of light in the vacuum) to calculate the distance travled by the backwards signal in each osciloscope, that is the position between of the short circuit

![Coaxial demo scheme](./coaxial-scheme-estimation.png)

In [15]:
offset_distances = [1.87, 0.91]

In [16]:
first_osc_after_open = 3

pos = [0]*len(oscs)

for i in range(len(oscs)):
    if i < first_pos_after_short-1: pos[i] = 0.7*3e8*t[i]/2 - offset_distances[i]
    else: pos[i] = 0.7*3e8*t[i]/2
    print('Osc {} Lenght from A to short circuit (m) = {}'.format(i+1,pos[i]))

Osc 1 Lenght from A to short circuit (m) = 10.404500000000002
Osc 2 Lenght from A to short circuit (m) = 10.3775
Osc 3 Lenght from A to short circuit (m) = 10.4055
Osc 4 Lenght from A to short circuit (m) = 10.3635


In [3]:
pos = np.array(pos)

In [8]:
print('{} ± {}'.format(pos.mean(), pos.std()/np.sqrt(len(oscs))))

10.38775 ± 0.008974791083919721


## Another test: Short circuit spotization

In this case we want to measure the location of a short circuirt, in another point of the same coaxial branch. The same steps have to be done.

In [ ]:
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'aut')
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'acq:type aver')
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'acq:count 128')
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'tim:scal 20 ns')
for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'tim:pos 60 ns')

As the fault is a short circuit, the backwards pulse is a minimum.

In [18]:
t = [0]*len(oscs)

for i in range(len(oscs)): 
    t[i] = float(instin.query(IPs[i], ports[i], oscs[i], 'meas:xmax? chan1'))
for i in range(len(oscs)):
    for i in range(len(oscs)): instin.write(IPs[i], ports[i], oscs[i], 'tim:pos 140 ns')
for i in range(len(oscs)):
    t[i] = float(instin.query(IPs[i], ports[i], oscs[i], 'meas:xmin? chan1')) - t[i]

for i in range(len(oscs)):
    print('Osc{} delay (s) = {}'.format(i+1, t[i]))

Osc1 delay (s) = 8.97e-08
Osc2 delay (s) = 8.060000000000001e-08
Osc3 delay (s) = 7.16e-08
Osc4 delay (s) = 7.19e-08


In [19]:
first_pos_after_short = 3

pos = [0]*len(oscs)

for i in range(len(oscs)):
    if i < first_pos_after_short-1: pos[i] = 0.7*3e8*t[i]/2 - offset_distances[i]
    else: pos[i] = 0.7*3e8*t[i]/2
    print('Osc {} Lenght from A to short circuit (m) = {}'.format(i+1,pos[i]))

Osc 1 Lenght from A to short circuit (m) = 7.5485
Osc 2 Lenght from A to short circuit (m) = 7.553000000000001
Osc 3 Lenght from A to short circuit (m) = 7.518000000000001
Osc 4 Lenght from A to short circuit (m) = 7.5495


In [9]:
pos = np.array(pos)

In [10]:
print('{} ± {}'.format(pos.mean(), pos.std()/np.sqrt(len(oscs))))

7.542250000000001 ± 0.007050044326101713


As the measurment has finished, the gateways can be closed

In [ ]:
for i in range(len(IPs)): insitn.close_term(IPs[i], ports[i])